In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import csv

In [ ]:
#Reading data file
raw_data = pd.read_csv('letter-recognition.data', sep=',',delimiter=None, header=None)

#Processing data as X_all and Y_all
X_all = raw_data.iloc[:,1:16]
Y_all = raw_data.iloc[:,0]

#Converting Y_all to hot vectors
converter = lambda x: ord(x)-ord('A')
Y_all = list(map(converter,Y_all.values.T.tolist()))
Y_all = pd.get_dummies(Y_all).values


In [ ]:
print(X_all.shape)
print(Y_all.shape)

In [ ]:
layer_info = [X_all.shape[1], 20, 30, Y_all.shape[1]]
layer_size = len(layer_info)
sess = tf.InteractiveSession()

with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, [None, 15], name = 'X_INPUT')
    Y = tf.placeholder(tf.float32, [None, 26], name = 'Y_INPUT')

def initialize_param(layer_info):
    weight_param = {}
    bias_param = {}
    for i in range(1,layer_size):
        weight_param['W'+str(i)] = tf.Variable(tf.truncated_normal([layer_info[i], layer_info[i-1]], stddev = 0.1))
        tf.summary.histogram('histogram', weight_param['W'+str(i)])
        bias_param['b'+str(i)] = tf.Variable(tf.constant(0.1, layer_info[i]))
        tf.summary.histogram('histogram', bias_param['b'+str(i)])
    return (weight_param, bias_param)
